In [2]:
import os
import re
import json
import shutil
import re
import logging
from math import radians, cos, sin, asin, sqrt

In [3]:
import pandas as pd
# from tqdm import tqdm
import numpy as np
import math

from sklearn.metrics import mean_squared_error,explained_variance_score
from sklearn.model_selection import KFold

import os
import glob
import warnings
warnings.filterwarnings('ignore')

In [4]:
!pip install geopandas
!pip install descartes
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple


In [5]:
os.chdir('/home/ma-user/work/FuSai/data/orderp/')
train_orders = []
for file in glob.glob('*.csv'):
    train_orders.append(file[0:len(file)-4])
train_orders = sorted(train_orders)
df_port_short = pd.read_csv('/home/ma-user/work/FuSai/data/port_R2ATest.csv')
train_gps_path = '/home/ma-user/work/FuSai/data/orderp/'
schedule_path = '/home/ma-user/work/FuSai/data/schedule/'
column_list = ['loadingOrder','timestamp','longitude','latitude','ds','speed','direction','vesselCurrentport','TRANSPORT_TRACE','time_used']

def geodistance(x,lng1,lat1,lng2,lat2):
    #lng1,lat1,lng2,lat2 = (120.12802999999997,30.28708,115.86572000000001,28.7427)
    lng1, lat1, lng2, lat2 = map(radians, [float(x[lng1]), float(x[lat1]), float(x[lng2]), float(x[lat2])]) # 经纬度转换成弧度
    dlon=lng2-lng1
    dlat=lat2-lat1
    a=sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    distance=2*asin(sqrt(a))*6371*1000 # 地球平均半径，6371km # unit: m
    distance=round(distance/1000,3) # unit: km
    return distance

In [6]:
len(train_orders)

17446

## 设计行程表(也就是分航段的特征如果某个订单有报错的话，直接跳到下一个i)

In [7]:
for i in range(len(train_orders)):
    train_df = pd.read_csv(train_gps_path+train_orders[i]+'.csv',names = column_list)
    df_schedule = pd.DataFrame(columns = ['loadingOrder','number','start_time','start_longitude','start_latitude','start_port','end_time'\
                  ,'end_longitude','end_latitude','end_port','target_longitude','target_latitude','target_port','ds','speed',\
                                          'stop_fraction','direction'])
    ttc = train_df[~(train_df['vesselCurrentport'].isna())]['vesselCurrentport'].unique()
    if (not all(train_df['TRANSPORT_TRACE'].isna())) and (type(train_df['TRANSPORT_TRACE'].unique()[0]) == str):
        transport_trace = train_df['TRANSPORT_TRACE'].unique()[0].split('-')
        if len(transport_trace) >= 2:
            df_port_ttc = df_port_short[df_port_short['TRANS_NODE_NAME'] == transport_trace[-1]]
            if df_port_ttc.shape[0] > 0:
                ttc1_lon = df_port_ttc['LONGITUDE'].values[0]
                ttc1_lat = df_port_ttc['LATITUDE'].values[0]
            else:
                ttc1_lon = train_df[(train_df['vesselCurrentport'] == ttc[-1])]['longitude'].mean()
                ttc1_lat = train_df[(train_df['vesselCurrentport'] == ttc[-1])]['latitude'].mean()
    if len(ttc) == 2 and (len(train_df['vesselCurrentport'].unique())>=3) and (train_df[(train_df['speed']>=2)].shape[0]\
                                                                               == train_df.shape[0]):
        train_df['vesselCurrentport'][(train_df['speed']>=2)] = np.nan
        df_schedule.loc[df_schedule.shape[0]] = [train_df['loadingOrder'].unique()[0],1,train_df['time_used'].iloc[0],\
                    train_df['longitude'].iloc[0],train_df['latitude'].iloc[0],ttc[0],\
                     train_df['time_used'].iloc[-1],train_df['longitude'].iloc[-1],train_df['latitude'].iloc[-1],ttc[-1],\
                    ttc1_lon,ttc1_lat,transport_trace[-1],train_df['ds'].sum(),\
                    train_df['speed'].mean(),0.,train_df['direction'].mean()]
    else:
        # train_df['vesselCurrentport'][(train_df['speed']>=2)] = np.nan
        # tpc = train_df[(train_df['vesselCurrentport'].isin(ttc))]['vesselCurrentport'].unique()
        for j in range(len(ttc)):
            # if ttc[j] not in tpc:
            #     continue
            # else:
            part = train_df[(train_df['vesselCurrentport'] == ttc[j])]
            # ind = np.where(tpc == ttc[j])[0][0]

            if j >= 1:
                train_partial = train_df[(train_df['time_used']>df_schedule['end_time'].iloc[-1])&(train_df['time_used']<part['time_used'].iloc[0])]
                if train_partial.shape[0] == 0:
                    continue
                else:
                    total_ds = train_partial['ds'].sum()
                    speed = train_partial['speed'].mean()
                    direction = train_partial['direction'].mean()
                    stop_fraction = train_partial[(train_partial['speed']<=1)].shape[0]/train_partial.shape[0]
                    df_schedule.loc[df_schedule.shape[0]] = [train_df['loadingOrder'].unique()[0],2*j-1,df_schedule['end_time'].iloc[-1],\
                        df_schedule['end_longitude'].iloc[-1],df_schedule['end_latitude'].iloc[-1],df_schedule['end_port'].iloc[-1],\
                        part['time_used'].iloc[0],part['longitude'].mean(),part['latitude'].mean(),part['vesselCurrentport'].iloc[0],\
                                                        ttc1_lon,ttc1_lat,transport_trace[-1],total_ds,speed,stop_fraction,direction]

            train_partial = train_df[(train_df['time_used']>part['time_used'].iloc[0])&(train_df['time_used']<part['time_used'].iloc[-1])]
            total_ds = part['ds'].sum()
            speed = part['speed'].mean()
            stop_fraction = part[(part['speed']<=1)].shape[0]/part.shape[0]
            direction = train_partial['direction'].mean()
            if ttc[j][0:5] == 'GHOST':
                ttc[j] = 'GHOST'+str(j)
                df_schedule.loc[df_schedule.shape[0]] = [train_df['loadingOrder'].unique()[0],2*j,part['time_used'].iloc[0],\
                        part['longitude'].mean(),part['latitude'].mean(),part['vesselCurrentport'].iloc[0],part['time_used'].iloc[-1],\
                         part['longitude'].mean(),part['latitude'].mean(),part['vesselCurrentport'].iloc[-1],ttc1_lon,ttc1_lat,\
                                                         transport_trace[-1],total_ds,speed,stop_fraction,direction]
            else:
                ttc_lon = df_port_short[df_port_short['TRANS_NODE_NAME'] == ttc[j]]['LONGITUDE'].values[0]
                ttc_lat = df_port_short[df_port_short['TRANS_NODE_NAME'] == ttc[j]]['LATITUDE'].values[0]
                df_schedule.loc[df_schedule.shape[0]] = [train_df['loadingOrder'].unique()[0],2*j,part['time_used'].iloc[0],ttc_lon,\
                             ttc_lat,part['vesselCurrentport'].iloc[0],part['time_used'].iloc[-1],ttc_lon,ttc_lat,\
                             part['vesselCurrentport'].iloc[-1],ttc1_lon,ttc1_lat,transport_trace[-1],total_ds,speed,\
                             stop_fraction,direction]

    df_schedule = df_schedule[df_schedule['number']>=0]
    df_schedule['route_dist'] = df_schedule.apply(lambda x:geodistance(x,"start_longitude","start_latitude",\
                                                                              "end_longitude","end_latitude"),axis=1)
    df_schedule['dist_to_dest'] = df_schedule.apply(lambda x:geodistance(x,"end_longitude","end_latitude",\
                                                                              "target_longitude","target_latitude"),axis=1)
    print('Finish processing the order =',train_orders[i],',i =',i,', the size of schedule is:',df_schedule['number'].shape[0])
    df_schedule.to_csv(schedule_path+train_orders[i]+'.csv',index=False)

Finish processing the order = AA191175561416 ,i = 0 , the size of schedule is: 11
Finish processing the order = AA236935066990 ,i = 1 , the size of schedule is: 7
Finish processing the order = AA500472614535 ,i = 2 , the size of schedule is: 1
Finish processing the order = AA597788695392 ,i = 3 , the size of schedule is: 13
Finish processing the order = AA642567891255 ,i = 4 , the size of schedule is: 5
Finish processing the order = AA681773043065 ,i = 5 , the size of schedule is: 3
Finish processing the order = AA849985976383 ,i = 6 , the size of schedule is: 9
Finish processing the order = AB232279245750 ,i = 7 , the size of schedule is: 7
Finish processing the order = AB272331390325 ,i = 8 , the size of schedule is: 1
Finish processing the order = AB283635056094 ,i = 9 , the size of schedule is: 5
Finish processing the order = AB309086787273 ,i = 10 , the size of schedule is: 3
Finish processing the order = AB341124011989 ,i = 11 , the size of schedule is: 7
Finish processing the or

Finish processing the order = AG789682008156 ,i = 100 , the size of schedule is: 9
Finish processing the order = AG881258910214 ,i = 101 , the size of schedule is: 5
Finish processing the order = AH140801152526 ,i = 102 , the size of schedule is: 3
Finish processing the order = AH212071198875 ,i = 103 , the size of schedule is: 15
Finish processing the order = AH294014104852 ,i = 104 , the size of schedule is: 7
Finish processing the order = AH533669291463 ,i = 105 , the size of schedule is: 3
Finish processing the order = AH607569652852 ,i = 106 , the size of schedule is: 9
Finish processing the order = AH611886522181 ,i = 107 , the size of schedule is: 7
Finish processing the order = AH792277811508 ,i = 108 , the size of schedule is: 5
Finish processing the order = AH856056393465 ,i = 109 , the size of schedule is: 7
Finish processing the order = AH861147242321 ,i = 110 , the size of schedule is: 17
Finish processing the order = AI101983760037 ,i = 111 , the size of schedule is: 7
Fi

Finish processing the order = AN738504256661 ,i = 201 , the size of schedule is: 9
Finish processing the order = AN852705658795 ,i = 202 , the size of schedule is: 3
Finish processing the order = AN886099352636 ,i = 203 , the size of schedule is: 3
Finish processing the order = AN886675756937 ,i = 204 , the size of schedule is: 7
Finish processing the order = AN910923566249 ,i = 205 , the size of schedule is: 7
Finish processing the order = AN935658391502 ,i = 206 , the size of schedule is: 7
Finish processing the order = AN970200600330 ,i = 207 , the size of schedule is: 7
Finish processing the order = AN976236735759 ,i = 208 , the size of schedule is: 5
Finish processing the order = AN986171507477 ,i = 209 , the size of schedule is: 3
Finish processing the order = AO236451541905 ,i = 210 , the size of schedule is: 15
Finish processing the order = AO255174617177 ,i = 211 , the size of schedule is: 15
Finish processing the order = AO275457362904 ,i = 212 , the size of schedule is: 11
F

Finish processing the order = AU207145267171 ,i = 300 , the size of schedule is: 9
Finish processing the order = AU242086092228 ,i = 301 , the size of schedule is: 5
Finish processing the order = AU250920147447 ,i = 302 , the size of schedule is: 3
Finish processing the order = AU265023696929 ,i = 303 , the size of schedule is: 15
Finish processing the order = AU372091858162 ,i = 304 , the size of schedule is: 11
Finish processing the order = AU381712713807 ,i = 305 , the size of schedule is: 5
Finish processing the order = AU419199344749 ,i = 306 , the size of schedule is: 5
Finish processing the order = AU452619319976 ,i = 307 , the size of schedule is: 7
Finish processing the order = AU468514204108 ,i = 308 , the size of schedule is: 9
Finish processing the order = AU474505484165 ,i = 309 , the size of schedule is: 7
Finish processing the order = AU489290890981 ,i = 310 , the size of schedule is: 7
Finish processing the order = AU582505472712 ,i = 311 , the size of schedule is: 9
Fi

Finish processing the order = BB352220256164 ,i = 401 , the size of schedule is: 3
Finish processing the order = BB468839282246 ,i = 402 , the size of schedule is: 5
Finish processing the order = BB481216170682 ,i = 403 , the size of schedule is: 5
Finish processing the order = BB481735750943 ,i = 404 , the size of schedule is: 3
Finish processing the order = BB483478271974 ,i = 405 , the size of schedule is: 7
Finish processing the order = BB530211078105 ,i = 406 , the size of schedule is: 9
Finish processing the order = BB606292699265 ,i = 407 , the size of schedule is: 9
Finish processing the order = BB655614310715 ,i = 408 , the size of schedule is: 3
Finish processing the order = BB703632984745 ,i = 409 , the size of schedule is: 11
Finish processing the order = BB737462916824 ,i = 410 , the size of schedule is: 3
Finish processing the order = BB762657408107 ,i = 411 , the size of schedule is: 5
Finish processing the order = BB846525971650 ,i = 412 , the size of schedule is: 5
Fin

...

Finish processing the order = LY342483685018 ,i = 7914 , the size of schedule is: 7
Finish processing the order = LY363398451050 ,i = 7915 , the size of schedule is: 5
Finish processing the order = LY392129868258 ,i = 7916 , the size of schedule is: 5
Finish processing the order = LY416025019497 ,i = 7917 , the size of schedule is: 3
Finish processing the order = LY439424512446 ,i = 7918 , the size of schedule is: 7
Finish processing the order = LY472382727933 ,i = 7919 , the size of schedule is: 7
Finish processing the order = LY507351823354 ,i = 7920 , the size of schedule is: 7
Finish processing the order = LY536032253024 ,i = 7921 , the size of schedule is: 5
Finish processing the order = LY618856598954 ,i = 7922 , the size of schedule is: 13
Finish processing the order = LY629536993286 ,i = 7923 , the size of schedule is: 7
Finish processing the order = LY688556303430 ,i = 7924 , the size of schedule is: 7
Finish processing the order = LY717980574692 ,i = 7925 , the size of schedu

Finish processing the order = MC411629059849 ,i = 8013 , the size of schedule is: 11
Finish processing the order = MC455199048870 ,i = 8014 , the size of schedule is: 5
Finish processing the order = MC540297591933 ,i = 8015 , the size of schedule is: 7
Finish processing the order = MC574344933715 ,i = 8016 , the size of schedule is: 5
Finish processing the order = MC574714462738 ,i = 8017 , the size of schedule is: 7
Finish processing the order = MC585291673304 ,i = 8018 , the size of schedule is: 7
Finish processing the order = MC612987633559 ,i = 8019 , the size of schedule is: 7
Finish processing the order = MC651227493249 ,i = 8020 , the size of schedule is: 5
Finish processing the order = MC670978211121 ,i = 8021 , the size of schedule is: 1
Finish processing the order = MC682544917191 ,i = 8022 , the size of schedule is: 7
Finish processing the order = MC726962158326 ,i = 8023 , the size of schedule is: 1
Finish processing the order = MC762262183105 ,i = 8024 , the size of schedu

Finish processing the order = MG531346423936 ,i = 8112 , the size of schedule is: 5
Finish processing the order = MG549404158373 ,i = 8113 , the size of schedule is: 7
Finish processing the order = MG555310488780 ,i = 8114 , the size of schedule is: 5
Finish processing the order = MG585497749379 ,i = 8115 , the size of schedule is: 7
Finish processing the order = MG585937015766 ,i = 8116 , the size of schedule is: 9
Finish processing the order = MG628389814043 ,i = 8117 , the size of schedule is: 5
Finish processing the order = MG659952350083 ,i = 8118 , the size of schedule is: 5
Finish processing the order = MG692023415781 ,i = 8119 , the size of schedule is: 7
Finish processing the order = MG701064043993 ,i = 8120 , the size of schedule is: 9
Finish processing the order = MG710267811230 ,i = 8121 , the size of schedule is: 3
Finish processing the order = MG797032329595 ,i = 8122 , the size of schedule is: 1
Finish processing the order = MG800043999841 ,i = 8123 , the size of schedul

Finish processing the order = MK148625633688 ,i = 8211 , the size of schedule is: 7
Finish processing the order = MK198307370205 ,i = 8212 , the size of schedule is: 5
Finish processing the order = MK203343932130 ,i = 8213 , the size of schedule is: 13
Finish processing the order = MK254014272680 ,i = 8214 , the size of schedule is: 3
Finish processing the order = MK260357064297 ,i = 8215 , the size of schedule is: 7
Finish processing the order = MK338882764456 ,i = 8216 , the size of schedule is: 7
Finish processing the order = MK349599079989 ,i = 8217 , the size of schedule is: 5
Finish processing the order = MK356974117124 ,i = 8218 , the size of schedule is: 11
Finish processing the order = MK436555288324 ,i = 8219 , the size of schedule is: 7
Finish processing the order = MK450748600188 ,i = 8220 , the size of schedule is: 9
Finish processing the order = MK455446954691 ,i = 8221 , the size of schedule is: 5
Finish processing the order = MK477654028079 ,i = 8222 , the size of sched

KeyboardInterrupt: 

In [13]:
# baseline只用到gps定位数据，即train_gps_path
train_gps_path = 'train0714.csv'
test_data_path = 'R2_ATest0711.csv'
order_data_path = 'loadingevents.csv'
port_data_path = 'port_fixed_0612.csv'

In [17]:
def get_data(data, mode = 'train'):
    assert mode=='train' or mode=='test'
    
    # data['timestamp'] = pd.to_datetime(data['timestamp'], infer_datetime_format=True)
    data['longitude'] = data['longitude'].astype(float)
    data['loadingOrder'] = data['loadingOrder'].astype(str)
    data['latitude'] = data['latitude'].astype(float)
    data['speed'] = data['speed'].astype(float)
    data['direction'] = data['direction'].astype(float)

    data["TRANSPORT_TRACE"] = data['TRANSPORT_TRACE'].astype(str)
    # data["Start"] = data["TRANSPORT_TRACE"].apply(lambda x: x.split("-")[0])
    if mode=='train':
        data['vesselNextportETA'] = pd.to_datetime(data['vesselNextportETA'], infer_datetime_format=True)
        # data["End"] = data["TRANSPORT_TRACE"].apply(lambda x: x.split("-")[-1])
        # data['StartEnd'] = data['Start'].values+'-'+data["End"].values
    elif mode=='test':
        data['temp_timestamp'] = data['timestamp']
        data['onboardDate'] = pd.to_datetime(data['onboardDate'], infer_datetime_format=True)

    print('========== Finish Collecting Data ==========')
    return data

In [26]:
test_schedule = pd.read_csv('/home/ma-user/work/FuSai/data/test_schedule.csv')
target_port = list(test_schedule['target_port'].unique())
target_port

['CLVAP',
 'MATNG',
 'SGSIN',
 'FRFOS',
 'INMAA',
 'JPHKT',
 'SVAQJ',
 'BHBAH',
 'MXZLO',
 'PAONX',
 'MYTPP',
 'ZADUR',
 'CVRAI',
 'CLSAI',
 'ARENA',
 'MACAS',
 'CMKBI',
 'GMBJL',
 'FRLEH',
 'NAWVB',
 'ESVAL',
 'GHTEM',
 'HKHKG',
 'GRPIR',
 'CRLIO',
 'HON',
 'EGPSD',
 'KRINC',
 'DZALG',
 'ESALG',
 'AEJEA',
 'UYMVD',
 'TNTUN',
 'DOCAU',
 'CAVAN',
 'PHBTG',
 'PKQCT',
 'ITGOA',
 'JPTYO',
 'PACOL',
 'DEHAM',
 'TTPOS',
 'RUVVO',
 'RTM',
 'BRSSZ',
 'CNYTN',
 'NGAPP',
 'BDCGP',
 'NZAKL',
 'GBFXT',
 'IDJKT',
 'CLSAN',
 'SIKOP',
 'CNSHK',
 'SNDKR',
 'LBBEY',
 'OMSOH',
 'GBSOU',
 'KWKWI',
 'INNSA',
 'MYPKG',
 'LKCMB',
 'TWKHH',
 'GTPRQ',
 'AUBNE',
 'ARBUE',
 'INMUN',
 'KHKOS',
 'CAHAL',
 'CIABJ',
 'PKBQM',
 'PAMIT',
 'HRRIJ',
 'SADMM',
 'THLCH']

In [19]:
test_data = pd.read_csv('/home/ma-user/work/FuSai/data/'+test_data_path)
course_list = list(sorted(test_data["TRANSPORT_TRACE"].unique()))
test_data["Port_list"] = test_data["TRANSPORT_TRACE"].apply(lambda x: x.split("-"))
start_list = test_data["TRANSPORT_TRACE"].apply(lambda x: x.split("-")[0])
test_data = get_data(test_data, mode='test')

========== Finish Collecting Data ==========


In [59]:
port_list = []
for i in range(len(np.unique(test_data['Port_list'].values))):
    port_list.extend(np.unique(test_data['Port_list'].values)[i])
port_list = sorted(np.unique(port_list))

In [60]:
port_list

['AEAUH',
 'AEJEA',
 'ANPHI',
 'ARBUE',
 'ARENA',
 'AUBNE',
 'BDCGP',
 'BHBAH',
 'BJCOO',
 'BRNVT',
 'BRPNG',
 'BRRIO',
 'BRSSZ',
 'CAHAL',
 'CAVAN',
 'CGPNR',
 'CIABJ',
 'CLSAI',
 'CLSAN',
 'CLVAP',
 'CMKBI',
 'CNDCB',
 'CNHKG',
 'CNNBG',
 'CNNSA',
 'CNSGH',
 'CNSHA',
 'CNSHK',
 'CNTAO',
 'CNXAM',
 'CNYTN',
 'COBUN',
 'COCTG',
 'CRLIO',
 'CVRAI',
 'DEHAM',
 'DOCAU',
 'DOHAI',
 'DZALG',
 'EGPSD',
 'EGPSE',
 'EGSUE',
 'EGSUZ',
 'ESALG',
 'ESBCN',
 'ESVAL',
 'FRFOS',
 'FRLEH',
 'FRMRS',
 'GALBV',
 'GBFXT',
 'GBSOU',
 'GHTEM',
 'GMBJL',
 'GPPTP',
 'GRPIR',
 'GTIZ4',
 'GTPRQ',
 'HKHKG',
 'HNPCR',
 'HON',
 'HRRIJ',
 'HUNGARY',
 'IDJKT',
 'INMAA',
 'INMUN',
 'INNSA',
 'ITGOA',
 'ITSPE',
 'JMKIN',
 'JPHKT',
 'JPTYO',
 'KHKOS',
 'KRINC',
 'KRPUS',
 'KWKWI',
 'LBBEY',
 'LKCMB',
 'LTKLJ',
 'MACAS',
 'MATNG',
 'MTMLA',
 'MUPLU',
 'MXLZC',
 'MXZLO',
 'MYPKG',
 'MYTPP',
 'NAWVB',
 'NGAPP',
 'NLRTM',
 'NZAKL',
 'OMSOH',
 'PABLB',
 'PACOL',
 'PAMIT',
 'PAONX',
 'PHBTG',
 'PKBQM',
 'PKKHI',
 'PKQCT',


In [45]:
train_df = pd.read_csv(train_gps_path+train_orders[17076]+'.csv',names = column_list)
df_schedule = pd.DataFrame(columns = ['loadingOrder','number','start_time','start_longitude','start_latitude','start_port','end_time'\
              ,'end_longitude','end_latitude','end_port','target_longitude','target_latitude','target_port','ds','speed',\
                                      'stop_fraction','direction'])

In [79]:
total_schedule = None
for i in range(0,len(train_orders)):
    df_schedule = pd.read_csv(schedule_path+train_orders[i]+'.csv')
    if (df_schedule['target_port'].unique() in target_port) or (any(df_schedule['end_port'].unique()) in port_list):
        df_odd = df_schedule[df_schedule['number']%2 == 1]
        speed = np.sum((df_odd['end_time'].values-df_odd['start_time'].values)*df_odd['speed'].values)/\
        np.sum(df_odd['end_time'].values-df_odd['start_time'].values)
        stop_fraction = np.sum((df_schedule['end_time'].values-df_schedule['start_time'].values)*df_schedule['stop_fraction'].values)/\
        np.sum(df_schedule['end_time'].values-df_schedule['start_time'].values)
        if df_odd.shape[0] == 0:
            df_schedule['label'] = df_schedule['end_time'].iloc[-1]-df_schedule['end_time']+3600*df_schedule['dist_to_dest'].iloc[-1]/\
                (speed * (1-stop_fraction**2))
        else:
            df_schedule['label'] = df_odd['end_time'].iloc[-1]-df_schedule['end_time']+3600*df_schedule['dist_to_dest'].iloc[-1]/\
                (speed * (1-stop_fraction**2))
        total_schedule = pd.concat([total_schedule,df_schedule])
        print('dist to dest:',df_schedule['dist_to_dest'].iloc[-1],',the speed is',speed,',the label is:',\
              3600*df_schedule['dist_to_dest'].iloc[-1] / (speed * (1-stop_fraction**2)))
        print('order = ',train_orders[i],',i =',i,', the size of total_schedule is:',total_schedule.shape)
total_schedule.to_csv('/home/ma-user/work/FuSai/data/train_schedule.csv',index=False)

dist to dest: 0.0 ,the speed is 35.90707428540889 ,the label is: 0.0
order =  AA191175561416 ,i = 0 , the size of total_schedule is: (11, 20)
dist to dest: 822.954 ,the speed is 31.274436090225567 ,the label is: 94730.22603678325
order =  AA500472614535 ,i = 2 , the size of total_schedule is: (12, 20)
dist to dest: 461.324 ,the speed is 27.018945812314005 ,the label is: 61857.70316383289
order =  AA597788695392 ,i = 3 , the size of total_schedule is: (25, 20)
dist to dest: 461.324 ,the speed is 24.747911552487334 ,the label is: 67726.53501669955
order =  AA642567891255 ,i = 4 , the size of total_schedule is: (30, 20)
dist to dest: 0.0 ,the speed is 26.847921225382933 ,the label is: 0.0
order =  AA681773043065 ,i = 5 , the size of total_schedule is: (33, 20)
dist to dest: 2549.004 ,the speed is 23.235759199287617 ,the label is: 397579.80334892514
order =  AA849985976383 ,i = 6 , the size of total_schedule is: (42, 20)
dist to dest: 146.281 ,the speed is 32.62538180877956 ,the label is: 

dist to dest: 6294.6759999999995 ,the speed is 27.283644983917775 ,the label is: 1067594.5173846732
order =  AI101983760037 ,i = 111 , the size of total_schedule is: (423, 20)
dist to dest: 2308.5860000000002 ,the speed is 23.65757172763241 ,the label is: 462610.7673147515
order =  AI143342019007 ,i = 112 , the size of total_schedule is: (430, 20)
dist to dest: 0.0 ,the speed is 28.338319919566597 ,the label is: 0.0
order =  AI238689352910 ,i = 113 , the size of total_schedule is: (439, 20)
dist to dest: 0.0 ,the speed is 29.216942179990397 ,the label is: 0.0
order =  AI338314636212 ,i = 114 , the size of total_schedule is: (444, 20)
dist to dest: 5007.83 ,the speed is 24.750792384433037 ,the label is: 857767.7473878923
order =  AI358169421638 ,i = 115 , the size of total_schedule is: (449, 20)
dist to dest: 0.0 ,the speed is 25.79629616503957 ,the label is: 0.0
order =  AI365973571487 ,i = 116 , the size of total_schedule is: (464, 20)
dist to dest: 0.0 ,the speed is 28.15507643992459

dist to dest: 0.0 ,the speed is 23.09090909090909 ,the label is: 0.0
order =  AN605049121722 ,i = 200 , the size of total_schedule is: (867, 20)
dist to dest: 0.0 ,the speed is 28.682433711751536 ,the label is: 0.0
order =  AN738504256661 ,i = 201 , the size of total_schedule is: (876, 20)
dist to dest: 12645.226 ,the speed is 23.370558375634516 ,the label is: 1947870.2644089686
order =  AN852705658795 ,i = 202 , the size of total_schedule is: (879, 20)
dist to dest: 0.0 ,the speed is 20.750356856585974 ,the label is: 0.0
order =  AO255174617177 ,i = 211 , the size of total_schedule is: (894, 20)
dist to dest: 9179.802 ,the speed is 27.910108334520622 ,the label is: 1230857.9861152156
order =  AO275457362904 ,i = 212 , the size of total_schedule is: (905, 20)
dist to dest: 3167.5570000000002 ,the speed is 31.291751054476943 ,the label is: 393438.1482195004
order =  AO287854813767 ,i = 213 , the size of total_schedule is: (924, 20)
dist to dest: 37.923 ,the speed is 21.006237328794136 ,

dist to dest: 0.0 ,the speed is 26.676368192785272 ,the label is: 0.0
order =  AU582505472712 ,i = 311 , the size of total_schedule is: (1308, 20)
dist to dest: 12455.228000000001 ,the speed is 20.473067045199105 ,the label is: 2341349.773203984
order =  AU768249118596 ,i = 312 , the size of total_schedule is: (1319, 20)
dist to dest: 0.0 ,the speed is 29.651239717672436 ,the label is: 0.0
order =  AU809825861449 ,i = 313 , the size of total_schedule is: (1332, 20)
dist to dest: 11661.274 ,the speed is 29.169516570766802 ,the label is: 1440155.2164721186
order =  AU851161490058 ,i = 314 , the size of total_schedule is: (1337, 20)
dist to dest: 14461.796 ,the speed is 24.736055365950794 ,the label is: 2337305.1024174658
order =  AU857831582677 ,i = 315 , the size of total_schedule is: (1352, 20)
dist to dest: 0.0 ,the speed is 30.649549158027433 ,the label is: 0.0
order =  AU974110730214 ,i = 316 , the size of total_schedule is: (1357, 20)
dist to dest: 16327.146 ,the speed is 25.823315

dist to dest: 0.0 ,the speed is 32.10020311442113 ,the label is: 0.0
order =  BB737462916824 ,i = 410 , the size of total_schedule is: (1794, 20)
dist to dest: 0.0 ,the speed is 30.9122806752161 ,the label is: 0.0
order =  BB762657408107 ,i = 411 , the size of total_schedule is: (1799, 20)
dist to dest: 0.0 ,the speed is 23.672206898212224 ,the label is: 0.0
order =  BB846525971650 ,i = 412 , the size of total_schedule is: (1804, 20)
dist to dest: 0.0 ,the speed is 25.138023473072884 ,the label is: 0.0
order =  BB975482080134 ,i = 418 , the size of total_schedule is: (1813, 20)
dist to dest: 0.0 ,the speed is 26.191396422575497 ,the label is: 0.0
order =  BB976986081018 ,i = 419 , the size of total_schedule is: (1820, 20)
dist to dest: 868.97 ,the speed is 23.60330578512397 ,the label is: 159654.91855931017
order =  BC159185419328 ,i = 420 , the size of total_schedule is: (1823, 20)
dist to dest: 0.0 ,the speed is 28.39260579701348 ,the label is: 0.0
order =  BC159365740735 ,i = 421 , 

...

dist to dest: 2958.742 ,the speed is 27.097902097902093 ,the label is: 393073.9527933235
order =  ZI715380147814 ,i = 16961 , the size of total_schedule is: (71303, 20)
dist to dest: 0.0 ,the speed is 29.797518475849106 ,the label is: 0.0
order =  ZI754901839359 ,i = 16962 , the size of total_schedule is: (71310, 20)
dist to dest: 2520.5660000000003 ,the speed is 24.085586215093524 ,the label is: 394010.99923747586
order =  ZI758704505909 ,i = 16963 , the size of total_schedule is: (71317, 20)
dist to dest: 0.0 ,the speed is 29.696132208413147 ,the label is: 0.0
order =  ZI846564157388 ,i = 16964 , the size of total_schedule is: (71326, 20)
dist to dest: 0.0 ,the speed is 24.893523501682335 ,the label is: 0.0
order =  ZI864782463424 ,i = 16966 , the size of total_schedule is: (71335, 20)
dist to dest: 12006.608999999999 ,the speed is 30.419908841999217 ,the label is: 1514607.2658534807
order =  ZI873821855333 ,i = 16967 , the size of total_schedule is: (71348, 20)
dist to dest: 6618.52

dist to dest: 0.0 ,the speed is 31.547745358090182 ,the label is: 0.0
order =  ZM100823470907 ,i = 17054 , the size of total_schedule is: (71687, 20)
dist to dest: 9390.326 ,the speed is 27.220187924214308 ,the label is: 1245673.4875644066
order =  ZM132977009554 ,i = 17055 , the size of total_schedule is: (71694, 20)
dist to dest: 9356.623 ,the speed is 30.247006450308973 ,the label is: 1116658.2779440787
order =  ZM143493257411 ,i = 17056 , the size of total_schedule is: (71701, 20)
dist to dest: 6735.981 ,the speed is 25.393371668635876 ,the label is: 993475.554314194
order =  ZM199847812307 ,i = 17057 , the size of total_schedule is: (71708, 20)
dist to dest: 8993.28 ,the speed is 27.761796334216427 ,the label is: 1255523.7100808818
order =  ZM240450564327 ,i = 17058 , the size of total_schedule is: (71719, 20)
dist to dest: 1138.85 ,the speed is 30.38121170864534 ,the label is: 135868.86238016852
order =  ZM269419926061 ,i = 17059 , the size of total_schedule is: (71722, 20)
dist 

dist to dest: 0.0 ,the speed is 32.04002238090391 ,the label is: 0.0
order =  ZP607491634506 ,i = 17141 , the size of total_schedule is: (72077, 20)
dist to dest: 0.0 ,the speed is 27.161683753063347 ,the label is: 0.0
order =  ZP741809502671 ,i = 17145 , the size of total_schedule is: (72082, 20)
dist to dest: 9908.253 ,the speed is 30.591354524668464 ,the label is: 1173065.9020628706
order =  ZP759338703737 ,i = 17146 , the size of total_schedule is: (72091, 20)
dist to dest: 6294.173000000001 ,the speed is 28.653550607288068 ,the label is: 993343.5160698218
order =  ZP844935729078 ,i = 17150 , the size of total_schedule is: (72098, 20)
dist to dest: 0.0 ,the speed is 36.57978144840707 ,the label is: 0.0
order =  ZP926671978079 ,i = 17152 , the size of total_schedule is: (72107, 20)
dist to dest: 0.0 ,the speed is 27.864546845938477 ,the label is: 0.0
order =  ZQ111245070636 ,i = 17153 , the size of total_schedule is: (72114, 20)
dist to dest: 0.0 ,the speed is 31.57556070277965 ,the

dist to dest: 4861.649 ,the speed is 24.69791170013341 ,the label is: 746815.9701367471
order =  ZT829024982407 ,i = 17247 , the size of total_schedule is: (72476, 20)
dist to dest: 0.0 ,the speed is 24.023831347387723 ,the label is: 0.0
order =  ZT901629475061 ,i = 17248 , the size of total_schedule is: (72479, 20)
dist to dest: 0.0 ,the speed is 30.011887072808314 ,the label is: 0.0
order =  ZU221599602866 ,i = 17254 , the size of total_schedule is: (72482, 20)
dist to dest: 7042.314 ,the speed is 27.33827474020815 ,the label is: 1341246.3494745109
order =  ZU222179672068 ,i = 17255 , the size of total_schedule is: (72489, 20)
dist to dest: 34.67 ,the speed is 29.344835047236984 ,the label is: 5080.925459732492
order =  ZU240441429268 ,i = 17256 , the size of total_schedule is: (72496, 20)
dist to dest: 0.0 ,the speed is 27.88874999099557 ,the label is: 0.0
order =  ZU272523978011 ,i = 17257 , the size of total_schedule is: (72503, 20)
dist to dest: 1637.358 ,the speed is 27.95823926

dist to dest: 0.0 ,the speed is 25.82165660993332 ,the label is: 0.0
order =  ZY278743564478 ,i = 17355 , the size of total_schedule is: (72885, 20)
dist to dest: 7111.945 ,the speed is 34.60975609756097 ,the label is: 753368.4603568364
order =  ZY279822510301 ,i = 17356 , the size of total_schedule is: (72888, 20)
dist to dest: 7664.655 ,the speed is 24.87827939651556 ,the label is: 1193199.3997894973
order =  ZY287341271768 ,i = 17358 , the size of total_schedule is: (72895, 20)
dist to dest: 0.0 ,the speed is 31.778938783961188 ,the label is: 0.0
order =  ZY294539775651 ,i = 17359 , the size of total_schedule is: (72904, 20)
dist to dest: 11023.452 ,the speed is 26.33206244504366 ,the label is: 1840753.1492923496
order =  ZY328733087007 ,i = 17360 , the size of total_schedule is: (72917, 20)
dist to dest: 0.0 ,the speed is 24.595098422507643 ,the label is: 0.0
order =  ZY478940010232 ,i = 17370 , the size of total_schedule is: (72924, 20)
dist to dest: 8751.413 ,the speed is 29.4453

In [73]:
df_schedule['target_port'].unique()

array(['GHOST7', 'GHOST6'], dtype=object)

In [42]:
total_schedule

,loadingOrder,number,start_time,start_longitude,start_latitude,start_port,end_time,end_longitude,end_latitude,end_port,target_longitude,target_latitude,target_port,ds,speed,stop_fraction,direction,route_dist,dist_to_dest,label
0,ZM703455429528,0,0.0,114.275347,22.577700,CNYTN,713.0,114.275347,22.577700,CNYTN,103.70462,1.303141,SGSIN,0.927634,2.800000,0.400000,23590.000000,0.000,2627.006,4.635134e+06
1,ZM703455429528,1,713.0,114.275347,22.577700,CNYTN,291929.0,103.786382,1.255955,SGSIN,103.70462,1.303141,SGSIN,2731.645262,37.443114,0.000000,21278.257485,2627.939,10.495,4.343918e+06
2,ZM703455429528,2,291929.0,103.704620,1.303141,SGSIN,378030.0,103.704620,1.303141,SGSIN,103.70462,1.303141,SGSIN,0.958739,0.031250,1.000000,15055.920000,0.000,0.000,4.257817e+06
3,ZM703455429528,3,378030.0,103.704620,1.303141,SGSIN,1375785.0,32.569650,29.826579,EGSUZ,103.70462,1.303141,SGSIN,8469.394271,28.437086,0.000000,26009.556291,8121.388,8121.388,3.260062e+06
4,ZM703455429528,4,1375785.0,32.556659,29.940875,EGSUZ,1391759.0,32.556659,29.940875,EGSUZ,103.70462,1.303141,SGSIN,0.681147,0.000000,1.000000,23487.933333,0.000,8124.504,3.244088e+06
5,ZM703455429528,5,1391759.0,32.556659,29.940875,EGSUZ,1625035.0,14.539832,35.818284,MTMLA,103.70462,1.303141,SGSIN,1905.665372,24.430380,0.000000,26239.607595,1801.466,9847.449,3.010812e+06
6,ZM703455429528,6,1625035.0,14.509457,35.896571,MTMLA,2755044.0,14.509457,35.896571,MTMLA,103.70462,1.303141,SGSIN,3.580609,0.083333,1.000000,20635.214286,0.000,9850.099,1.880803e+06
7,ZM703455429528,7,2755044.0,14.509457,35.896571,MTMLA,3063779.0,3.117636,36.788492,DZALG,103.70462,1.303141,SGSIN,1084.070123,34.130435,0.000000,28804.326087,1024.516,10860.499,1.572068e+06
8,ZM703455429528,8,3063779.0,3.064607,36.768455,DZALG,3178870.0,3.064607,36.768455,DZALG,103.70462,1.303141,SGSIN,6.813580,0.065934,0.989011,18231.344444,0.000,10865.471,1.456977e+06
